In [24]:
import torch as th

dtype = th.float64  # Use float32 if you are on GeForce GPU

gpuid = 0
device = th.device("cuda:"+ str(gpuid))
#device = th.device("cpu")  # un-comment to change back to CPU

print("Execution device: ",device)
print("PyTorch version: ", th.__version__ )
print("CUDA available: ", th.cuda.is_available())
print("CUDA version: ", th.version.cuda)
print("CUDA device:", th.cuda.get_device_name(gpuid))

Execution device:  cuda:0
PyTorch version:  1.0.1
CUDA available:  True
CUDA version:  10.0.130
CUDA device: GeForce RTX 2080 Ti


In [25]:
# Utility functions

# return the lower triangle of A in column order i.e. vech(A)
def vech(A):
    count = 0
    c = A.shape[0]
    v = th.zeros(c * (c + 1) // 2, device=device, dtype=dtype)
    for j in range(c):
        for i in range(j,c):
            v[count] = A[i,j]
            count += 1
    return v

# vech2L   create lower triangular matrix L from vechA
def vech2L(v,n):
    count = 0
    L = th.zeros((n,n), device=device, dtype=dtype)
    for j in range(n):
        for i in range(j,n):
            L[i,j]=v[count]
            count += 1
    return L

In [26]:
def matrix_elements(n, vechLk, vechLl, Sym, Mass, vecQ):
    '''
    Returns: a dictionary with the skl, tkl, vkl matrix elements
    n : the size of the Lk matrices
    vechL(k,l): vector of parameters use for Lk and Ll matrices
    Sym: a single symmetry projection matrix
    Mass: a matrix of "reduced mass" values for the particles needed for tkl
    vecQ: an array of products of particle charges needed for vkl
    '''
    # build Lk and Ll

    Lk = vech2L(vechLk,n);
    Ll = vech2L(vechLl,n);

    # apply symmetry projection on Ll

    # th.t() is shorthand for th.transpose(X, 0,1)
    PLl = th.t(Sym) @ Ll;

    # build Ak, Al, Akl, invAkl

    Ak = Lk@th.t(Lk);
    Al = PLl@th.t(PLl);
    Akl = Ak+Al
    invAkl = th.inverse(Akl);

    # Overlap (normalized)
    skl = 2**(3*n/2) * th.sqrt( th.pow(th.abs(th.det(Lk))*th.abs(th.det(Ll))/th.det(Akl) ,3) );

    # Kinetic energy

    tkl = skl*(6*th.trace(Mass@Ak@invAkl@Al));

    # potential energy

    TWOoSqrtPI = 1.1283791670955126 # 2/sqrt(pi)

    RIJ = th.zeros((n,n), device=device, dtype=dtype);
    # 1/rij i~=j
    for j in range(0,n-1):
        for i in range(j+1,n):
            tmp2 = invAkl[i,i] + invAkl[j,j] - 2*invAkl[i,j];
            #RIJ[i,j] = TWOoSqrtPI * skl/th.sqrt(tmp2);
            RIJ[i,j] = 1/th.sqrt(tmp2)

    # 1/rij i=j
    for i in range(0,n):
        #RIJ[i,i] = TWOoSqrtPI * skl/th.sqrt(invAkl[i,i]);
        RIJ[i,i] = 1/th.sqrt(invAkl[i,i])

    RIJ = TWOoSqrtPI*skl*RIJ
    Q = vech2L(vecQ,n);

    vkl = th.sum(RIJ*Q)

    return {'skl':skl, 'tkl':tkl, 'vkl':vkl}

In [27]:
def test_matrix_elements():
# test input with a known result
    n = 3;
    vechLk = th.tensor([ 1.00000039208682, 0.02548044275764261, 0.3525161612610669,
                         1.6669144815242515, 0.9630555318946559, 1.8382882034659822
                       ], device=device, dtype=dtype, requires_grad=True);

    vechLl = th.tensor([ 1.3353550436464964, 0.9153272033682132, 0.7958636766525028,
                         1.8326931436447955, 0.3450426931160630, 1.8711839323167831
                       ], device=device, dtype=dtype, requires_grad=True);

    Sym = th.tensor([[0,0,1],
                    [0,1,0],
                    [1,0,0]], device=device, dtype=dtype);

    Mass = th.tensor([[5.446170e-4, 2.723085077e-4, 2.723085077e-4],
                     [2.723085077e-4, .5002723085, 2.723085077e-4],
                     [2.723085077e-4, 2.723085077e-4, .5002723085 ]], device=device, dtype=dtype);

    vecQ = th.tensor([1, -1, -1, -1, 1, -1], device=device, dtype=dtype);

    matels = matrix_elements(n, vechLk, vechLl, Sym, Mass, vecQ)

    print('skl: ',matels['skl']) # should be 0.5334
    print('tkl: ',matels['tkl']) # should be 4.3509
    print('vkl: ',matels['vkl']) # should be -2.3840

In [28]:
start_time = time.time()
test_matrix_elements()
print(" took {:.6f} seconds ".format(time.time() - start_time))

skl:  tensor(0.5334, device='cuda:0', dtype=torch.float64, grad_fn=<MulBackward0>)
tkl:  tensor(4.3509, device='cuda:0', dtype=torch.float64, grad_fn=<MulBackward0>)
vkl:  tensor(-2.3840, device='cuda:0', dtype=torch.float64, grad_fn=<SumBackward0>)
 took 0.028487 seconds 


In [29]:
def energy(x,n,nb,Mass,Charge,Sym,symc):
    '''
    Returns: the energy at the point (parameters) x
    n: number of particles defined in input
    nb: number of basis functions
    Mass: matrix of "reduced" mass constants for system
    Charge: vector of "charge products" for particle pairs
    Sym: a tensor containing the symmetry projection matrices
    symc: a vector of coefficients for the symmetry projection terms
    '''

    nx = len(x);
    nn = int(n*(n+1)/2);
    nsym = len(symc);

    # extract linear coefs "eigen vector" from x
    c = x[-nb:];
    # reshape non-linear variables for easier indexing
    X = th.reshape(x[:nb*nn], (nb,nn))

    # Init H S T V
    H = th.zeros((nb,nb), device=device, dtype=dtype);
    S = th.zeros((nb,nb), device=device, dtype=dtype);
    T = th.zeros((nb,nb), device=device, dtype=dtype);
    V = th.zeros((nb,nb), device=device, dtype=dtype);

    # outer loop is over symmetry terms
    for k in range(0,nsym):

        for j in range(0,nb):
            for i in range(j,nb):

                vechLi = X[i,:]
                vechLj = X[j,:]

                matels = matrix_elements(n,vechLi,vechLj,Sym[:,:,k],Mass,Charge);

                S[i,j] += symc[k]*matels['skl'];
                T[i,j] += symc[k]*matels['tkl'];
                V[i,j] += symc[k]*matels['vkl'];

    H = T + V

    # complete upper triangle of H and S
    for i in range(0,nb):
        for j in range(i+1,nb):
            H[i,j] = H[j,i];
            S[i,j] = S[j,i];

    # The energy from the Rayleigh quotient

    cHc = c@H@c;
    cSc = c@S@c;
    eng = cHc/cSc;

    return eng

In [30]:
def test_energy():

    Mass = th.tensor([[0.5, 0.0, 0.0],
                     [0.0, 0.5, 0.0],
                     [0.0, 0.0, 0.5]], device=device, dtype=dtype);

    Charge = th.tensor([-3, 1, 1, -3, 1, -3], device=device, dtype=dtype);

    # symmetry projection terms
    Sym = th.zeros((3,3,6), device=device, dtype=dtype)
    # (1)(2)(3)
    Sym[:,:,0] = th.tensor([[1,0,0],[0,1,0],[0,0,1]], device=device, dtype=dtype);
    # (12)
    Sym[:,:,1] = th.tensor([[0,1,0],[1,0,0],[0,0,1]], device=device, dtype=dtype);
    # (13)
    Sym[:,:,2] = th.tensor([[0,0,1],[0,1,0],[1,0,0]], device=device, dtype=dtype);
    # (23)
    Sym[:,:,3] = th.tensor([[1,0,0],[0,0,1],[0,1,0]], device=device, dtype=dtype);
    # (123)
    Sym[:,:,4] = th.tensor([[0,1,0],[0,0,1],[1,0,0]], device=device, dtype=dtype);
    # (132)
    Sym[:,:,5] = th.tensor([[0,0,1],[1,0,0],[0,1,0]], device=device, dtype=dtype);

    # coeff's
    symc = th.tensor([4.0,4.0,-2.0,-2.0,-2.0,-2.0], device=device, dtype=dtype);

    n=3;
    nb=8;

    xvechL=th.tensor([
         1.6210e+00, -2.1504e-01,  9.0755e-01,  9.7866e-01, -2.8418e-01,
        -3.5286e+00, -3.3045e+00, -4.5036e+00, -3.2116e-01, -7.1901e-02,
         1.5167e+00, -8.4489e-01, -2.1377e-01, -3.6127e-03, -5.3774e-03,
        -2.1263e+00, -2.5191e-01,  2.1235e+00, -2.1396e-01, -1.4084e-03,
        -1.0092e-02,  4.5349e+00,  9.4837e-03,  1.1225e+00, -2.1315e-01,
         5.8451e-02, -4.9410e-03,  5.0853e+00,  7.3332e-01,  5.0672e+00,
        -2.1589e-01, -6.8986e-03, -1.4310e-02,  1.5979e+00,  3.3946e-02,
        -8.7965e-01, -1.1121e+00, -2.1903e-03, -4.6925e-02,  2.1457e-01,
         3.3045e-03,  4.5120e+00, -2.1423e-01, -1.6493e-02, -2.3429e-03,
        -8.6715e-01, -6.7070e-02,  1.5998e+00
     ], device=device, dtype=dtype, requires_grad=False)

    evec = th.tensor([
      -6.0460e-02,  7.7708e-05, 1.6152e+00,  9.5443e-01,
      1.1771e-01,  3.2196e+00,  9.6344e-01, 3.1398e+00
    ], device=device, dtype=dtype, requires_grad=False)

    # join the parameters into x1
    x1 = th.tensor(th.cat((xvechL,evec)), device=device, dtype=dtype, requires_grad=True)

    eng = energy(x1,n,nb,Mass,Charge,Sym,symc)
    print(eng) # Should return -7.3615 at this point x1
    return x1

In [31]:
start_time = time.time()
xrestart = test_energy()
print(" took {} seconds ".format(time.time() - start_time))

/home/peirong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


tensor(-7.3615, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
 took 1.922145128250122 seconds 


In [32]:
def opt_energy(steps=1, num_basis=8):

    ############################################################################
    # Input constants for Li atom (infinite nuclear mass)
    ############################################################################
    Mass = th.tensor([[0.5, 0.0, 0.0],
                     [0.0, 0.5, 0.0],
                     [0.0, 0.0, 0.5]], device=device, dtype=dtype);

    Charge = th.tensor([-3, 1, 1, -3, 1, -3], device=device, dtype=dtype);

    # symmetry projection terms
    Sym = th.zeros((3,3,6), device=device, dtype=dtype)
    # (1)(2)(3)
    Sym[:,:,0] = th.tensor([[1,0,0],[0,1,0],[0,0,1]], device=device, dtype=dtype);
    # (12)
    Sym[:,:,1] = th.tensor([[0,1,0],[1,0,0],[0,0,1]], device=device, dtype=dtype);
    # (13)
    Sym[:,:,2] = th.tensor([[0,0,1],[0,1,0],[1,0,0]], device=device, dtype=dtype);
    # (23)
    Sym[:,:,3] = th.tensor([[1,0,0],[0,0,1],[0,1,0]], device=device, dtype=dtype);
    # (123)
    Sym[:,:,4] = th.tensor([[0,1,0],[0,0,1],[1,0,0]], device=device, dtype=dtype);
    # (132)
    Sym[:,:,5] = th.tensor([[0,0,1],[1,0,0],[0,1,0]], device=device, dtype=dtype);

    # coeff's
    symc = th.tensor([4.0,4.0,-2.0,-2.0,-2.0,-2.0], device=device, dtype=dtype);
    ############################################################################


    n=3
    nb=num_basis
    th.manual_seed(3)
    x1 = th.empty(int(nb*n*(n+1)/2 + nb), device=device, dtype=dtype, requires_grad=True)
    th.nn.init.uniform_(x1, a=-.8, b=.8)

    #x1 = xrestart

    optimizer = th.optim.Rprop([x1], lr=0.001, etas=(0.5, 1.2), step_sizes=(1e-06, 50))

    for i in range(steps):
        optimizer.zero_grad()
        loss = energy(x1,n,nb,Mass,Charge,Sym,symc)
        loss.backward()
        optimizer.step()

        if (i<10 or not i%10):print('step: {:5}  f: {:4.12f}  gradNorm: {:.9f}'.format(i, loss, th.norm(x1.grad)))
    print('step: {:5}  f: {:4.12f}  gradNorm: {:.9f}'.format(i, loss, th.norm(x1.grad)))
    return x1

In [33]:
start_time = time.time()
xrestart = opt_energy(steps=100, num_basis=8)
print(" took {} seconds ".format(time.time() - start_time))

step:     0  f: -1.877050436926  gradNorm: 4.966496125
step:     1  f: -1.901530279386  gradNorm: 4.508943568
step:     2  f: -1.928853761720  gradNorm: 4.169625466
step:     3  f: -1.959106181524  gradNorm: 4.075478768
step:     4  f: -1.994104439752  gradNorm: 4.160535426
step:     5  f: -2.036738475116  gradNorm: 4.157667488
step:     6  f: -2.087847091161  gradNorm: 4.131654279
step:     7  f: -2.148828898876  gradNorm: 4.128423259
step:     8  f: -2.221449884604  gradNorm: 4.118043760
step:     9  f: -2.307299335622  gradNorm: 4.086703701
step:    10  f: -2.407812246465  gradNorm: 4.026246863
step:    20  f: -4.456750747555  gradNorm: 1.957011951
step:    30  f: -6.674216437869  gradNorm: 0.881610320
step:    40  f: -6.998994035212  gradNorm: 0.405565188
step:    50  f: -7.160233084773  gradNorm: 0.287270188
step:    60  f: -7.227468977848  gradNorm: 0.123267445
step:    70  f: -7.276917554524  gradNorm: 0.135205009
step:    80  f: -7.302988371468  gradNorm: 0.106376258
step:    9